In [1]:
from langchain import OpenAI
from langchain.agents import Tool, tool
from langchain import agents
import langchain
import os
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import sys
import dziner

langchain.debug = False
# from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI

from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
Embedding_model = 'text-embedding-3-large' 

from langchain.text_splitter import CharacterTextSplitter

In [2]:
import dziner.sascorer
from rdkit import Chem
from rdkit.Chem import QED
from rdkit.Chem import Descriptors

@tool
def check_validity(smiles: str):
    '''
    This tool inputs a SMILES and outputs chemical feasibility, SA scores, molecular weight and the smiles
    '''
    smiles = smiles.replace("\n", "")
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return "Invalid SMILES", 0, 0, smiles

    # Calculate SA score
    sa_score = dziner.sascorer.calculateScore(mol)

    # Calculate QED score
    qed_score = QED.qed(mol)
    molecular_weight = Descriptors.MolWt(mol)

    return "Valid SMILES", sa_score, molecular_weight, smiles

# Example usage
test_smiles = "CC(=O)OC1=CC=CC=C1C(=O)O"
check_validity(test_smiles)

('Valid SMILES',
 1.580039750008826,
 180.15899999999996,
 'CC(=O)OC1=CC=CC=C1C(=O)O')

In [4]:
## This model is based on this paper: https://pubs.acs.org/doi/10.1021/jacs.2c11420 fine-tuned on the hmof dataset

from dziner.surrogates.MOFormer.CO2_adsorption import predict_CO2_adsorption_with_uncertainty
import torch

@tool
def predict_CO2_adsoprtion(smiles):
    '''
    This tool predicts the mean CO2 Adsorption and the standard deviation in prediction for an input SMILES.
    This model is based on this paper: https://pubs.acs.org/doi/10.1021/jacs.2c11420 fine-tuned on the hmof dataset.
    '''
    smiles = smiles.replace("\n", "")
    mean_prediction, std_prediction = predict_CO2_adsorption_with_uncertainty(smiles)    
    return np.round(mean_prediction, 3), np.round(std_prediction, 3)
initial_smiles = 'COc1cc(C(=O)[O-])c(c(c1c1ccc(cc1)c1ccc(cc1)C(=O)[O-])OC)[O]'
predict_CO2_adsoprtion(initial_smiles)

(1.443, 0.135)

In [5]:
# from dziner.surrogates.CO_2.model import predict_single_point

# def predict_CO2_adsoprtion(smiles):
#     '''
#     This tool predicts the mean CO2 Adsorption and it standard deviation for a SMILES.
#     This model is based on this paper: https://pubs.acs.org/doi/10.1021/jacs.2c11420 fine-tuned on the hmof dataset
#     '''
#     base_model_path = "../dziner/surrogates/CO_2/hmof_finetuned_models/hmof_finetuned_{}.pth"
    
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     predictions = []

#     # Looping through the ensemble of models with 5-fold cross validation.
#     for cvid in range(5):  
#         model_path = base_model_path.format(cvid)
#         print(model_path)
#         inference_model = torch.load(model_path)
# #         try:
#         # prediction = predict_single_point(smiles, model_path, device)
#         prediction = SMILES_to_property(smiles, model=inference_model)
#         predictions.append(prediction)
# #         except:
# #             predictions.append(None)
    
#     valid_predictions = [p for p in predictions if p is not None]
    
#     if not valid_predictions:
#         return 'Invalid molecule', None
    
#     mean_prediction = np.mean(valid_predictions)
#     std_prediction = np.std(valid_predictions)
    
#     return np.round(mean_prediction, 3), np.round(std_prediction, 3)

In [6]:
# initial_smiles = '[O-]C(=O)c1ccc(cc1)c1nnc(nn1)c1ccc(cc1)C(=O)[O-]'
# predict_CO2_adsoprtion(initial_smiles)

In [7]:
from langchain.memory import ConversationBufferMemory

RetrievalQA_prompt = """What are the design guidelines for the making the organic linker adsorb CO2 better? 
    This can be based on making changes to the elements or the functional groups or other changes to the molecule. Summarize your answer and cite paper
    by the title, DOI and the journal and year the document was published on."""

@tool
def lookup_papers(prompt):
    '''Useful for getting chemical intuition for the problem.
    This tool looks up design guidelines for organic linkers with higher CO2 adsorption by looking through research papers.
    It also includes information on the paper citation or DOI.
    '''
    guide_lines = []
    for m in range(4):
        text_splitter = CharacterTextSplitter(
            chunk_size=500, chunk_overlap=20)
        paper_file =f'../data/papers/CO2_adsorption/{m}.pdf'
        pages = PyPDFLoader(paper_file).load_and_split()
        sliced_pages = text_splitter.split_documents(pages)
        faiss_vectorstore = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
        
        llm=ChatOpenAI(
                        model_name='gpt-4o',
                        temperature=0.1,
                        )
        g = dziner.RetrievalQABypassTokenLimit(faiss_vectorstore, RetrievalQA_prompt, llm)
        guide_lines.append(g)
        # break
    return " ".join(guide_lines)


# guidel_lines = lookup_papers("")
# guidel_lines

In [8]:
tools = [lookup_papers, check_validity, predict_CO2_adsoprtion]

tool_names = [tool.name for tool in tools]  
tool_desc = [tool.description for tool in tools]
initial_organic_linker = "Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N" 
Human_prompt = f"Make changes to the organic linker {initial_organic_linker} to increase its CO2 adsroption. Suggest 20 new candidates."

input_data = {
            "input": Human_prompt,
            "prompt": RetrievalQA_prompt,
            "tools": tools,
            "tool_names": tool_names,
            "tool_desc": tool_desc
        }

In [9]:
from dziner.agents import dZiner

FORMAT_INSTRUCTIONS = """
Use the following format:

    Thought: you should always think about what to do and never leave this empty
    Action: the action to take, should be one of the tools [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    You don't always need to use a tool if you know the answer.
    Thought: I know the final answer
    Final Answer: the final answer to the input question.
    The Final Answer MUST come in JSON format with NO extra text.
    
    Example final answer format:

     "Iteration": 0,
      "SMILES": "CC(=O)OC1=CC=C(C=C1)C(=O)N2CCN(CC2)CC3=CC=CC=C3I",
      "Modification": The change made to the SMILES in the previous iteration and the long detailed reasoning behind it.
      "CO2 Adsorption": 1.0,
      "SA Score": 2.01,
      "Molecular Weight": 150,
      "Validity": "Valid"

    Report all iterations, even if the SMILES is invalid. An iteration includes suggesting a new SMILES, evaluating its CO2 Adsorption, and determining its validity.
    
    Use the exact SMILES without any newlines or extra parentheses. Do not repeat SMILES that already exist in the iteration history.
    
    You have a final answer once 20 new SMILES candidates are generated and evaluated. 
       

"""

SUFFIX = """Start by:

1. Researching Design Guidelines: Summarize the guidelines in a few bullet points, with citations (e.g., paper DOI) included for any design guidelines used.

2. Evaluating Initial SMILES:

CO2 Adsorption: Assess the CO2 Adsorption of the initial SMILES.
Validity: Check the validity of the initial SMILES structure.
3. Manually Modifying SMILES:

Manual Adjustments: Based on the guidelines from Step 1, make manual changes to the SMILES string. These changes may involve adding, removing, or replacing elements, functional groups, or rings in the molecule’s core.
Apply different modifications that could help increase CO2 Adsorption without significantly increasing molecular size/weight.
Alignment with Guidelines: Justify in great details how each new SMILES aligns with the design guidelines, such as adding aromatic rings if suggested.
4. Evaluating Modified SMILES:

CO2 Adsorption: Assess the CO2 Adsorption of each new SMILES and see if the change you made step 3 is favorable.
Validity: Ensure each new SMILES is valid.

Choose the SMILES with the highest CO2 Adsorption.
If no improvement is seen, revert to the previous valid SMILES in the iteration with the best CO2 Adsorption and retry Steps 3–4.
Iteration Details:

Iteration 0: Use the initial input SMILES.
Iteration Limit: MUST only stop after generating 20 new SMILES candidates (20 SMILES MUST be in your final answer).
Molecular Weight: Prioritize changes that does not increase the molecular weight too much.
Final Steps:

Document All Changes: Even if reverted, all changes must be documented and reported in the end given the format instructions.
Avoid Redundancy: Do not repeat SMILES in the iteration history.
Begin with Problem Description:
Question: {input}
Thought: {agent_scratchpad}
"""

In [10]:
from dziner.agents import dZiner
from dziner.prompts import PREFIX

from langchain_anthropic import ChatAnthropic
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=key, temperature=0.3)

from langchain_openai import ChatOpenAI
# model = ChatOpenAI(
#                     temperature=0.1,
#                     model_name='chatgpt-4o-latest',
#                     request_timeout=1000,
#                     max_tokens=8192,
#                 )

agent = dZiner(tools, property="CO2 Adsorption",
               model='gpt-4o', verbose=True, temp=0.3,
              suffix=SUFFIX, format_instructions=FORMAT_INSTRUCTIONS,
              max_tokens=8192 , max_iterations=120, n_design_iterations=25).agent


agent.get_prompts

<bound method Runnable.get_prompts of AgentExecutor(memory=ConversationBufferMemory(output_key='output', input_key='input', memory_key='chat_history'), verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='You are a helpful Chemist AI assistant called dZiner. You are tasked to make changes to a molecule based on some design guidelines\n    and optimize the its CO2 Adsorption.\n    Always use the tools to get chemical intuition learn about the design guidelines. If you don\'t know, do not make up answers.\n    Explain changes you make to the molecule in details at each step but do not put redundant information and make it short.\n    \n\nlookup_papers(prompt) - Useful for getting chemical intuition for the problem.\n    This tool looks up design guidelines for organic linkers with higher CO2 adsorption by looking through research papers.\n    It also includes information o

In [11]:
from dziner.agents import dZiner
from dziner.prompts import PREFIX

from langchain_anthropic import ChatAnthropic
# model = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=key, temperature=0.3)

from langchain_openai import ChatOpenAI
# model = ChatOpenAI(
#                     temperature=0.1,
#                     model_name='chatgpt-4o-latest',
#                     request_timeout=1000,
#                     max_tokens=8192,
#                 )

agent = dZiner(tools, property="CO2 Adsorption",
               model='gpt-4o', verbose=True, temp=0.3,
              suffix=SUFFIX, format_instructions=FORMAT_INSTRUCTIONS,
              max_tokens=8192 , max_iterations=120, n_design_iterations=25).agent


agent.get_prompts

<bound method Runnable.get_prompts of AgentExecutor(memory=ConversationBufferMemory(output_key='output', input_key='input', memory_key='chat_history'), verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='You are a helpful Chemist AI assistant called dZiner. You are tasked to make changes to a molecule based on some design guidelines\n    and optimize the its CO2 Adsorption.\n    Always use the tools to get chemical intuition learn about the design guidelines. If you don\'t know, do not make up answers.\n    Explain changes you make to the molecule in details at each step but do not put redundant information and make it short.\n    \n\nlookup_papers(prompt) - Useful for getting chemical intuition for the problem.\n    This tool looks up design guidelines for organic linkers with higher CO2 adsorption by looking through research papers.\n    It also includes information o

In [12]:

from langchain.callbacks import get_openai_callback
import contextlib
import sys

langchain.debug = False
test = 3
output_file_path = f"results/2.MOF-CO2-adsorption/CO2-20-it({test})_log.txt"

class Tee:
    def __init__(self, *streams):
        self.streams = streams

    def write(self, data):
        for stream in self.streams:
            stream.write(data)
            stream.flush()

    def flush(self):
        for stream in self.streams:
            stream.flush()

f = open(output_file_path,'w')

tee = Tee(sys.stdout,f)

with contextlib.redirect_stdout(tee):
    with get_openai_callback() as cb:
        response = agent.invoke(input_data)

    print(cb)

f.close()



> Entering new AgentExecutor chain...
Thought: To begin, I need to research design guidelines for organic linkers with higher CO2 adsorption. This will help me understand the structural features that enhance CO2 adsorption.

Action: lookup_papers
Action Input: Design guidelines for organic linkers with higher CO2 adsorption.
Observation: To design organic linkers that adsorb CO2 better, the following guidelines can be considered:

1. **Functional Group Introduction**: Introducing functional groups such as -OSO3H, -NO2, -OH, -NH2, and other polar groups like -N=C(R)-, -N=N- can significantly enhance CO2 adsorption due to their ability to interact strongly with CO2 molecules.

2. **Pore Size Optimization**: Adjusting the pore size and window to match the kinetic diameter of CO2 can improve selective adsorption. This involves precise engineering of the pore structures to enhance the interaction potential between the adsorbent and CO2.

3. **Acidity-Basicity Balance**: Since CO2 is an ac

[17:06:26] SMILES Parse Error: syntax error while parsing: Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OCH3)
[17:06:26] SMILES Parse Error: Failed parsing SMILES 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OCH3)' for input: 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OCH3)'


The modified SMILES with a methoxy group is invalid. I will revert to the previous best modification (trifluoromethyl group) and continue making further modifications.

### Iteration 12:
**Modification**: Introduce a hydroxyl group (-OH) to the aromatic ring to increase polarity and interaction with CO2.

Action: check_validity
Action Input: Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OH)
Observation: ('Invalid SMILES', 0, 0, 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OH)')
Thought:

[17:06:28] SMILES Parse Error: syntax error while parsing: Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OH)
[17:06:28] SMILES Parse Error: Failed parsing SMILES 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OH)' for input: 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(OH)'


The modified SMILES with a hydroxyl group is invalid. I will revert to the previous best modification (trifluoromethyl group) and continue making further modifications.

### Iteration 13:
**Modification**: Introduce a methyl group (-CH3) to the aromatic ring to increase interaction with CO2.

Action: check_validity
Action Input: Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(C)
Observation: ('Valid SMILES', 3.399612494276653, 405.41400000000016, 'Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(C)')
Thought:The modified SMILES is valid with a synthetic accessibility (SA) score of 3.40 and a molecular weight of 405.41. Next, I will predict its CO2 adsorption.

Action: predict_CO2_adsoprtion
Action Input: Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)N)C(=O)[O-])N(C)
Observation: (1.056, 0.075)
Thought:The modified SMILES with a methyl group has a CO2 adsorption value of 1.056 with a standard deviation of 0.075, which is higher than the initial value but lower tha

In [13]:
import json
import pandas as pd

output_str = response['output'].replace("```json", "")[:-3]#.replace("\\n", "").replace("\n", "").replace(" ", "").replace("''", "")

# output_str = output_str.replace("''", "")
output_dict = json.loads(output_str)
df = pd.DataFrame(output_dict)
df

,Iteration,SMILES,Modification,CO2 Adsorption,SA Score,Molecular Weight,Validity
0,0,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Initial SMILES,1.036,3.36,391.39,Valid
1,1,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a hydroxyl group (-OH) to the aroma...,0.997,3.50,407.39,Valid
2,2,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced an amino group (-NH2) to the aromat...,0.984,3.46,406.40,Valid
3,3,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a nitro group (-NO2) to the aromati...,0.909,3.61,436.38,Valid
4,4,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a sulfonic acid group (-SO3H) to th...,0.898,3.49,471.45,Valid
5,5,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a carboxyl group (-COOH) to the aro...,0.970,3.37,435.40,Valid
6,6,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a fluorine atom to the aromatic ring.,1.522,3.59,409.38,Valid
7,7,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a chlorine atom to the aromatic ring.,1.176,3.60,425.83,Valid
8,8,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced an alkylamine group (-NHCH3) to the...,1.000,3.59,420.43,Valid
9,9,Nc1cc(C(=O)[O-])c(cc1c1ccc(cc1)c1c(N)cc(c(c1N)...,Introduced a trifluoromethyl group (-CF3) to t...,1.819,3.58,459.38,Valid


In [14]:
csv_file_path = f"results/2.MOF-CO2-adsorption/CO2-20-it({test}).csv"

df.to_csv(csv_file_path, index=False)

In [ ]:
# import matplotlib.pyplot as plt

# iterations = [item['iteration'] for item in iteration_data]
# mean_scores = [item['data']['Docking Score'] for item in iteration_data]
# # std_band_gaps = [item['data']['STD CMC'] for item in iteration_data]
# smiles_labels = [item['data']['SMILES'] for item in iteration_data]
# chemical_feasibilities = [item['data']['Chemical Feasibility'] for item in iteration_data]

# plt.figure(dpi=400)

# colors = ['C2' if feas == 'Valid' else 'C0' for feas in chemical_feasibilities]

# # Plot with error bars and color-coded points
# plt.figure(dpi=400)
# for i, (x, y, yerr, color) in enumerate(zip(iterations, mean_scores, np.zeros(len(mean_scores)), colors)):
#     plt.errorbar(x, y, yerr=yerr, fmt='o', color=color, ecolor='k', capsize=3,
#                  linestyle='None', alpha=0.6, markersize=10, label=chemical_feasibilities[i])



# vertical_offsets = [(-10 + 5 * (i % 2)) for i in range(len(iterations))]  # Alternating vertical offsets

# for x, y, label, offset in zip(iterations, mean_scores, smiles_labels, vertical_offsets):
#     plt.text(x-0.2, y + offset/100, label, verticalalignment='bottom', horizontalalignment='right', fontsize=4, color='C0', rotation=90)

# ax=plt.gca()
# handles, labels = ax.get_legend_handles_labels()
# by_label = dict(zip(labels, handles))

# plt.legend(by_label.values(), by_label.keys(), loc="lower left")

# plt.xlabel('Agent Iterations')
# plt.xlim([-0.5,7])
# plt.ylim([-11, -5])
# plt.ylabel('Docking Score')

# # plt.savefig(f"../results/{iteration_data[0]['data']['SMILES']}_guided.png")

In [ ]:
## Plotting sequntially for presentation purposes

# from rdkit import Chem
# from rdkit.Chem import Draw
# import matplotlib.pyplot as plt



# from rdkit.Chem import Draw
# from PIL import Image

# def visualize_smiles(smiles, filename):
#     # Convert SMILES to molecule
#     mol = Chem.MolFromSmiles(smiles)
    
#     # Generate image of the molecule with RDKit and a transparent background
#     img = Draw.MolToImage(mol, size=(1000, 1000), kekulize=True, wedgeBonds=True, options=Draw.DrawingOptions())
    
#     # Convert the image to have a transparent background
#     img = img.convert("RGBA")
#     datas = img.getdata()
#     newData = []
#     for item in datas:
#         # Change all white (also shades of whites)
#         # to transparent
#         if item[0] > 200 and item[1] > 200 and item[2] > 200:
#             newData.append((255, 255, 255, 0))
#         else:
#             newData.append(item)
#     img.putdata(newData)
    
#     # Save the image
#     img.save(filename, "PNG")


# import json


# with open(f"../results/CMC/{initial_surfactant}_guided.json", "r") as json_file:
#     iteration_data = json.load(json_file)

# iterations = [item['iteration'] for item in iteration_data]
# mean_cmcs = [item['data']['CMC'] for item in iteration_data]
# std_cmcs = np.zeros(len(mean_cmcs))
# smiles_labels = [item['data']['SMILES'] for item in iteration_data]
# chemical_feasibilities = [item['data']['Chemical Feasibility'] for item in iteration_data]

# plt.figure(dpi=400)

# colors = ['C2' if feas == 'Valid' else 'C0' for feas in chemical_feasibilities]

# # Plot with error bars and color-coded points

# iter = 0

# plt.figure(dpi=400)
# for i, (x, y, yerr, color) in enumerate(zip(iterations, mean_cmcs, np.zeros(len(mean_cmcs)), colors)):
#     plt.errorbar(x, y, yerr=yerr, fmt='o', color=color, ecolor='k', capsize=3,
#                  linestyle='None', alpha=0.6, markersize=10, label=chemical_feasibilities[i])

#     filename = f'../results/CMC/iter_{i}_{smiles_labels[i]}.png'
#     visualize_smiles(smiles_labels[i], filename)
#     if i == iter:
#         break


# vertical_offsets = [(-10 + 5 * (k % 2)) for k in range(len(iterations))]  # Alternating vertical offsets

# for i, (x, y, label, offset) in enumerate(zip(iterations, mean_cmcs, smiles_labels, vertical_offsets)):
#     plt.text(x-0.2, y + offset/100, label, verticalalignment='bottom', horizontalalignment='right', fontsize=7, color='C0', rotation=90)
#     if i == iter:
#         break

# ax=plt.gca()
# handles, labels = ax.get_legend_handles_labels()
# by_label = dict(zip(labels, handles))

# plt.legend(by_label.values(), by_label.keys(), loc="lower left")

# plt.xlabel('Agent Iterations')
# plt.xlim([-0.5,11])
# plt.ylim([-1, 5])
# plt.ylabel('logCMC (μM)')

# plt.savefig(f"../results/CMC/{iter}_{smiles_labels[iter]}_guided.png")

In [ ]:
# from rdkit import Chem
# from rdkit.Chem import Draw
# import matplotlib.pyplot as plt



# from rdkit.Chem import Draw
# from PIL import Image

# def visualize_smiles(smiles, filename):
#     # Convert SMILES to molecule
#     mol = Chem.MolFromSmiles(smiles)
    
#     # Generate image of the molecule with RDKit and a transparent background
#     img = Draw.MolToImage(mol, size=(1000, 1000), kekulize=True, wedgeBonds=True, options=Draw.DrawingOptions())
    
#     # Convert the image to have a transparent background
#     img = img.convert("RGBA")
#     datas = img.getdata()
#     newData = []
#     for item in datas:
#         # Change all white (also shades of whites)
#         # to transparent
#         if item[0] > 200 and item[1] > 200 and item[2] > 200:
#             newData.append((255, 255, 255, 0))
#         else:
#             newData.append(item)
#     img.putdata(newData)
    
#     # Save the image
#     img.save(filename, "PNG")

# # Example usage
# filename = f'{initial_surfactant}.png'
# visualize_smiles(initial_surfactant, filename)